# Prepara o Dataset de Treinamento - sem usar toda a arquitetura em ação, apenas coletando os tweeters

Essa parte o time de DataScience ou MachineLearning podem fazer em paralelo sem influenciar na arquitetura de streaming,
e o time de engenharia de dados apenas fornece um dataset de treinamento

## Funcionamento para a criação do DataSet:

* Passo 1) Executar o sricpt logwitter.py (você pode definir por quanto tempo ele vai executar)
* Passo 2) Executar o script Dataset.py (onde irá armazenar os eventos em um json local - quantos você desejar)
* Passo 3) Executar o Treinamento Abaixo

## Treinamento com arquivo ja pronto com as tags

In [13]:
import nltk
import re
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
from sklearn.model_selection import cross_val_predict

In [14]:
dataset = pd.read_csv('Data/Tweets_Mg.csv', encoding='utf-8')

In [15]:
dataset.head(2)

,Unnamed: 0,Created At,Text,Geo Coordinates.latitude,Geo Coordinates.longitude,User Location,Username,User Screen Name,Retweet Count,Classificacao,...,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24
0,0,Sun Jan 08 01:22:05 +0000 2017,���⛪ @ Catedral de Santo Antônio - Governador ...,NaN,NaN,Brasil,Leonardo C Schneider,LeoCSchneider,0,Neutro,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,Sun Jan 08 01:49:01 +0000 2017,"� @ Governador Valadares, Minas Gerais https:/...",-41.9333,-18.85,NaN,Wândell,klefnews,0,Neutro,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Realizar um pre-processamento basico:
* Remove linhas duplicadas na base de dados
* Remove Stopwords
* Fazer Stemming nos dados
* Remover caracteres indesejados como links, pontuação etc.

In [16]:
dataset.drop_duplicates(['Text'], inplace=True)

### Separar Texto e Classes

In [17]:
tweets = dataset['Text']
classes = dataset['Classificacao']

In [18]:
tweets[0]

'���⛪ @ Catedral de Santo Antônio - Governador Valadares/MG https://t.co/JSbKamIqUJ'

In [19]:
import nltk
nltk.download('stopwords')
nltk.download('rslp')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\samfs\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package rslp to
[nltk_data]     C:\Users\samfs\AppData\Roaming\nltk_data...
[nltk_data]   Package rslp is already up-to-date!


True

## Utilizar as funcoes abaixa para classificar e treinar e ajustar os textos

In [20]:
def RemoveStopWords(instancia):
    stopwords = set(nltk.corpus.stopwords.words('portuguese'))
    palavras = [i for i in instancia.split() if not i in stopwords]
    return (" ".join(palavras))
def Stemming(instancia):
    stemmer = nltk.stem.RSLPStemmer()
    palavras = []
    for w in instancia.split():
        palavras.append(stemmer.stem(w))
    return (" ".join(palavras))
def Limpeza_dados(instancia):
    # remove links, pontos, virgulas,ponto e virgulas dos tweets
    instancia = re.sub(r"http\S+", "", instancia).lower().replace('.','').replace(';','').replace('-','').replace(':','').replace(')','')
    return (instancia)

In [21]:
def Preprocessing(instancia):
    stemmer = nltk.stem.RSLPStemmer()
    instancia = re.sub(r"http\S+", "", instancia).lower().replace('.','').replace(';','').replace('-','').replace(':','').replace(')','')
    stopwords = set(nltk.corpus.stopwords.words('portuguese'))
    palavras = [stemmer.stem(i) for i in instancia.split() if not i in stopwords]
    return (" ".join(palavras))

# Aplica a função em todos os dados:
tweets = [Preprocessing(i) for i in tweets]

--------------------------------------------------------------------------------------------------------------------------------------------------------------------------
------------------------------------------------------------------------------------------------------

## Criar o modelo - Random Forest e NB

In [22]:
vectorizer = CountVectorizer(analyzer = "word", tokenizer = None, preprocessor = None,
                             stop_words = None, max_features = 5000)


train_data_features  = vectorizer.fit_transform(tweets)

modelo = MultinomialNB()
modelo.fit(freq_tweets,classes)


NameError: name 'freq_tweets' is not defined

In [ ]:
from sklearn.ensemble import RandomForestClassifier
forest = RandomForestClassifier(n_estimators = 100)
forest

In [ ]:
class_sentimentos = dataset["Classificacao"].values
forest = forest.fit(train_data_features, class_sentimentos)
forest

In [ ]:
print (modelo.classes_)

### Realizar Testes basicos

In [ ]:
# textos criados
testes = ['Esse Ano ta muito top, cada vez mais eu amo ele, todo dia otimo',
          'Odeio esse tal de flamengo, sem condicoes'
         ]

In [ ]:
# Pre-processar cada texto
testes = [Preprocessing(i) for i in testes]
freq_testes = vectorizer.transform(testes)

In [ ]:
# Fazendo a classificação com o modelo treinado.
for t, c in zip (testes,modelo.predict(freq_testes)):
    print (t +", "+ c)

In [ ]:
## Random forest
test_data_features = vectorizer.transform(testes)
resultados = forest.predict(test_data_features)
resultados

In [ ]:
resultados = cross_val_predict(modelo, freq_tweets, classes, cv = 10)
resultados

# Quão acurada é a média do modelo?
metrics.accuracy_score(classes, resultados)

In [ ]:
# Medidas de validação do modelo
sentimentos = ["Positivo", "Negativo", "Neutro"]
print(metrics.classification_report(classes, resultados, sentimentos))

In [ ]:
# Matriz de confusão
print(pd.crosstab(classes, resultados, rownames = ["Real"], colnames=["Predito"], margins=True))

--------------------------------------------------------------------------------------------------------------------------------------------------------------------------
------------------------------------------------------------------------------------------------------

## Naive Bayes

In [23]:
import re
from nltk.tokenize import word_tokenize
from string import punctuation 
from nltk.corpus import stopwords 


def processTweetBayes(tweet):
    tweet = tweet.lower()
    tweet = re.sub('((www\.[^\s]+)|(https?://[^\s]+))', 'URL', tweet)
    tweet = re.sub('@[^\s]+', 'AT_USER', tweet) 
    tweet = re.sub(r'#([^\s]+)', r'\1', tweet) 
    stopwords = set(nltk.corpus.stopwords.words('portuguese'))
    palavras = [word for word in tweet.split() if word not in stopwords]
    return (palavras)


def createProcessTweets(df):
    processdTweet = []
    for index,row in df.iterrows():
        processdTweet.append((processTweetBayes(row[2]),row[9]))
    return processdTweet

In [24]:
# Criar o trainiginSet:

trainingSet =  createProcessTweets(dataset)

In [25]:
trainingSet[0]

(['���⛪',
  '@',
  'catedral',
  'santo',
  'antônio',
  '-',
  'governador',
  'valadares/mg',
  'URL'],
 'Neutro')

In [26]:
### Construindo o vocabulario
import nltk 

def buildVocabulary(preprocessedTrainingData):
    all_words = []
    
    for (words, sentiment) in preprocessedTrainingData:
        all_words.extend(words)

    wordlist = nltk.FreqDist(all_words)
    word_features = wordlist.keys()
    
    return word_features

def extract_features(tweet):
    tweet_words = set(tweet)
    features = {}
    for word in word_features:
        features['contains(%s)' % word] = (word in tweet_words)
    return features 

In [27]:
word_features = buildVocabulary(trainingSet)
trainingFeatures = nltk.classify.apply_features(extract_features, trainingSet)

In [28]:
trainingFeatures

[({'contains(���⛪)': True, 'contains(@)': True, 'contains(catedral)': True, 'contains(santo)': True, 'contains(antônio)': True, 'contains(-)': True, 'contains(governador)': True, 'contains(valadares/mg)': True, 'contains(URL)': True, 'contains(�)': False, 'contains(valadares,)': False, 'contains(minas)': False, 'contains(gerais)': False, 'contains(��)': False, 'contains(���)': False, 'contains(psol)': False, 'contains(vai)': False, 'contains(questionar)': False, 'contains(aumento)': False, 'contains(vereadores)': False, 'contains(prefeito)': False, 'contains(bh)': False, 'contains(justiça)': False, 'contains(politica)': False, 'contains(estado)': False, 'contains(")': False, 'contains(bom)': False, 'contains(bandido)': False, 'contains(morto")': False, 'contains(deputado)': False, 'contains(cabo)': False, 'contains(júlio)': False, 'contains(condenado)': False, 'contains(fica)': False, 'contains(inelegível)': False, 'contains(10)': False, 'contains(anos)': False, 'contains("..e)': False

In [29]:
NBayesClassifier = nltk.NaiveBayesClassifier.train(trainingFeatures)

## Testar nosso Modelo

In [ ]:
# textos criados
testes = ['Esse Ano ta muito top, cada vez mais eu amo ele, todo dia otimo',
          'Odeio esse tal de flamengo, sem condicoes'
         ]

In [ ]:
def PreprocessingNewTweet(list_tweet):

    def processTweetBayes(tweet):
        tweet = tweet.lower()
        tweet = re.sub('((www\.[^\s]+)|(https?://[^\s]+))', 'URL', tweet)
        tweet = re.sub('@[^\s]+', 'AT_USER', tweet) 
        tweet = re.sub(r'#([^\s]+)', r'\1', tweet) 
        stopwords = set(nltk.corpus.stopwords.words('portuguese'))
        palavras = [word for word in tweet.split() if word not in stopwords]
        return (palavras)


    def createProcessTweets(tweets):
        processdTweet = []
        for tweet in tweets:
            processdTweet.append((processTweetBayes(tweet)))
        return processdTweet
    
    return createProcessTweets(list_tweet)

In [ ]:
TestDataset = PreprocessingNewTweet(testes)
TestDataset

In [ ]:
NBResultLabels = [NBayesClassifier.classify(extract_features(tweet)) for tweet in TestDataset]

In [ ]:
NBResultLabels

--------------------------------------------------------------------------------------------------------------------------------------------------------------------------
------------------------------------------------------------------------------------------------------

## Ler arquivo com os Twitters:

In [ ]:
import json
import glob
import re

texts = []
files = glob.glob(os.getcwd() + '\Data' + '\*.json')

for file in files:
    with open(file, 'r') as json_file:
        tweets = json.load(json_file)
        for tweet in tweets:
            try:
                obj = json.loads(tweet)
                texts.append(obj['text'])
            except:
                pass


In [ ]:
texts[0:2]

## Classificar os Tweets Coletados

In [ ]:
StreamingTweets = PreprocessingNewTweet(texts)

In [ ]:
NBResultLabels = [NBayesClassifier.classify(extract_features(tweet)) for tweet in StreamingTweets]

In [ ]:
for tweet,sentimental in zip(texts,NBResultLabels):
    print(f'Tweet = {tweet} ---> Sentimento: {sentimental} \n')

--------------------------------------------------------------------------------------------------------------------------------------------------------------------------
------------------------------------------------------------------------------------------------------

In [ ]:
############ ESSA FUNCAO PARA EXTRAIR E LIMPAR OS TEXTOS:
"""
import re
from nltk.tokenize import word_tokenize
from string import punctuation 
from nltk.corpus import stopwords 

def PreprocessingNewTweet(list_tweet):

    def processTweetBayes(tweet):
        tweet = tweet.lower()
        tweet = re.sub('((www\.[^\s]+)|(https?://[^\s]+))', 'URL', tweet)
        tweet = re.sub('@[^\s]+', 'AT_USER', tweet) 
        tweet = re.sub(r'#([^\s]+)', r'\1', tweet) 
        stopwords = set(nltk.corpus.stopwords.words('portuguese'))
        palavras = [word for word in tweet.split() if word not in stopwords]
        return (palavras)


    def createProcessTweets(tweets):
        processdTweet = []
        for tweet in tweets:
            processdTweet.append((processTweetBayes(tweet)))
        return processdTweet
    
    return createProcessTweets(list_tweet)

"""

# Tutorial de como salvar seu modelo em Pickle

## Primeiramente o estilo de tweet event para se adequar ao modelo:

* 1) Receba o event tweet
* 2) Realize o filtro de coletar apenas o text
* 3) Passe o texto para a funcao de limpar e extrair as palavras
* 4) Passe pelo modelo: NBResultLabels = NBayesClassifier.classify(extract_features(tweet))

In [ ]:
# TO save
import pickle
f = open('my_classifier.pickle', 'wb')
pickle.dump(NBayesClassifier, f)
f.close()

In [ ]:
# To Load
import pickle
f = open('my_classifier.pickle', 'rb')
classifier = pickle.load(f)
f.close()

In [ ]:
## Testando arquivo pickle

NBResultLabels = [classifier.classify(extract_features(tweet)) for tweet in TestDataset]
NBResultLabels

In [30]:
word_features

dict_keys(['���⛪', '@', 'catedral', 'santo', 'antônio', '-', 'governador', 'valadares/mg', 'URL', '�', 'valadares,', 'minas', 'gerais', '��', '���', 'psol', 'vai', 'questionar', 'aumento', 'vereadores', 'prefeito', 'bh', 'justiça', 'politica', 'estado', '"', 'bom', 'bandido', 'morto"', 'deputado', 'cabo', 'júlio', 'condenado', 'fica', 'inelegível', '10', 'anos', '"..e', '25%', 'mineiros', 'dizem', 'torcer', 'time', 'nenhum,mesmo', 'dentro', 'atlético-mg', 'cruzeiro.', 'pq?.."', '"a', 'gigantesca', 'barba', 'mal"', 'destaque', 'caderno', 'cultura', 'minas.', '"bb', 'governo', 'travam', 'disputa', 'sobre', 'depósitos', 'judiciais"', '"com', 'vcs', 'pequena!"', 'belo', 'horizonte', '(pron.', '[bɛloɾiˈzõntʃi][10])', 'capital', 'mg,', 'área', 'aproximadamente', '331', 'km²', '"daí', 'gente', 'visita', 'valadares"', '"que', 'lugar', 'eh', 'mãe?"', '"mg,', 'uns', 'parente', 'la"', 'mano', 'onde', 'q', '"é', 'bonita', 'bonita..."', 'latergram', 'ibituruna', 'home', '"erro', 'desconhecido"', 'm

In [31]:
type(word_features)

dict_keys

In [38]:
with open("word_feats", "wb") as f:
    data = str(trainingSet)
    f.write(data.encode("UTF-8"))

In [36]:
trainingSet


[(['���⛪',
   '@',
   'catedral',
   'santo',
   'antônio',
   '-',
   'governador',
   'valadares/mg',
   'URL'],
  'Neutro'),
 (['�', '@', 'governador', 'valadares,', 'minas', 'gerais', 'URL'], 'Neutro'),
 (['��', '@', 'governador', 'valadares,', 'minas', 'gerais', 'URL'], 'Neutro'),
 (['���', 'URL'], 'Neutro'),
 (['���',
   'psol',
   'vai',
   'questionar',
   'aumento',
   'vereadores',
   'prefeito',
   'bh',
   'justiça',
   '-',
   'politica',
   '-',
   'estado',
   'minas',
   'URL'],
  'Negativo'),
 (['"',
   'bom',
   'bandido',
   'morto"',
   'deputado',
   'cabo',
   'júlio',
   'condenado',
   'fica',
   'inelegível',
   '10',
   'anos',
   '-',
   'politica',
   '-',
   'estado',
   'minas',
   'URL'],
  'Neutro'),
 (['"..e',
   '25%',
   'mineiros',
   'dizem',
   'torcer',
   'time',
   'nenhum,mesmo',
   'dentro',
   'estado',
   'atlético-mg',
   'cruzeiro.',
   'pq?.."',
   'URL'],
  'Neutro'),
 (['"a',
   'gigantesca',
   'barba',
   'mal"',
   'destaque',
   'ca

In [37]:
type(trainingSet)

list